In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

print(np.__version__)
print(pd.__version__)
import sys
print(sys.version)
print(sns.__version__)

1.19.2
1.1.3
3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]
0.11.0


In [13]:
df = pd.read_csv("D:\\repos\\APDSProject\\MLTuning\\modeling\\CWE_Inputs.csv")
df.shape
df.columns
#df['Phase'].value_counts()
df['Weakness Abstraction'].value_counts()

Base        29058
Class       22371
Variant      1965
Compound     1776
Pillar        906
Name: Weakness Abstraction, dtype: int64

In [21]:
df = pd.read_csv("D:\\repos\\APDSProject\\MLTuning\\datacleanup\\cvss_final_dataset.csv")
df.head()
df.shape
df.info()
df.columns



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67113 entries, 0 to 67112
Data columns (total 43 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   timestamp                             67113 non-null  object 
 1   data_type                             67113 non-null  object 
 2   data_format                           67113 non-null  object 
 3   data_version                          67113 non-null  float64
 4   data_meta_ID                          67113 non-null  object 
 5   data_meta_ASSIGNER                    67113 non-null  object 
 6   problemtype_value                     67113 non-null  object 
 7   reference_data_url                    67113 non-null  object 
 8   reference_data_refsource              66712 non-null  object 
 9   description                           67113 non-null  object 
 10  cvssV3_version                        67113 non-null  float64
 11  cvssV3_vectorSt

In [8]:
def my_tokenizer(text):
    return text.split(' ') if text != None else []

tokens = df['description'].map(my_tokenizer).sum()



In [13]:
valsLower = [item.lower() for item in tokens]

In [15]:
from spacy.lang.en.stop_words import STOP_WORDS 

def remove_stopwords(tokens):
    return [t for t in valsLower if t not in STOP_WORDS]

In [16]:
from collections import Counter
counter = Counter(remove_stopwords(valsLower))

In [19]:
counter.most_common(100)

[('vulnerability', 36690),
 ('allows', 25256),
 ('attacker', 21935),
 ('remote', 20871),
 ('attackers', 14675),
 ('arbitrary', 14569),
 ('code', 14526),
 ('access', 14368),
 ('versions', 12585),
 ('crafted', 11482),
 ('service', 11220),
 ('user', 11085),
 ('issue', 11083),
 ('allow', 10894),
 ('cause', 10632),
 ('oracle', 10574),
 ('denial', 9617),
 ('windows', 9379),
 ('affected', 9361),
 ('execute', 8908),
 ('server', 8474),
 ('prior', 7945),
 ('information', 7556),
 ('sd', 7516),
 ('successful', 7356),
 ('version', 7263),
 ('web', 7104),
 ('data', 7061),
 ('exploit', 6635),
 ('file', 6440),
 ('discovered', 6274),
 ('users', 6154),
 ('attacks', 6118),
 ('exists', 6051),
 ('local', 6036),
 ('earlier', 5942),
 ('cvss', 5808),
 ('execution', 5600),
 ('function', 5580),
 ('memory', 5267),
 ('aka', 5140),
 ('malicious', 5042),
 ('buffer', 5002),
 ('unauthorized', 4925),
 ('network', 4837),
 ('ibm', 4710),
 ('system', 4686),
 ('application', 4474),
 ('result', 4472),
 ('unspecified', 4415)

In [ ]:
#tokens.remove('SCOPE')
tokens = [ elem for elem in tokens if elem != 'SCOPE']
tokens = [ elem for elem in tokens if elem != 'IMPACT']
tokens = [ elem for elem in tokens if elem != 'NOTE']
tokens = [ elem for elem in tokens if elem != 'Access Control']
tokens = [ elem for elem in tokens if elem != 'Authentication']
tokens = [ elem for elem in tokens if elem != 'Availability']
tokens = [ elem for elem in tokens if elem != 'Confidentiality']
tokens = [ elem for elem in tokens if elem != 'Integrity']
tokens = [ elem for elem in tokens if elem != 'Non-Repudiation']
tokens = [ elem for elem in tokens if elem != 'Other']

In [ ]:
from collections import Counter

counter = Counter(tokens)
counter.most_common(20)

# Remove Unwanted Columns

In [ ]:

remove_list = ['timestamp','data_type','data_format', 'data_version','data_meta_ASSIGNER','cvssV3_version','cvssV3_vectorString','cvssV2_version','cvssV2_vectorString']

df.drop(remove_list,axis = 1, inplace = True)
df.info
df.shape



# Prepare CVSS3 Data Frame




In [ ]:
df.columns
cvss3_features = ['cvssV3_attackVector', 'cvssV3_attackComplexity', 'cvssV3_privilegesRequired', 'cvssV3_userInteraction', 'cvssV3_scope','cvssV3_confidentialityImpact', 'cvssV3_integrityImpact','cvssV3_availabilityImpact', 'cvssV3_baseScore', 'cvssV3_baseSeverity',      'baseMetricV3_exploitabilityScore', 'baseMetricV3_impactScore', ]

cvss3_df = df[cvss3_features]

cvss3_df.columns
cvss3_df.info
cvss3_df.shape


# Extract the numerical and categorical columns from CVSS Version 3 Featue Set

In [ ]:
cvssv3_numerical_col = list(cvss3_df.describe().columns)
cvssv3_categorical_col = list(set(cvss3_df.columns).difference(cvssv3_numerical_col))

cvssv3_numerical_col


In [ ]:
cvss3_df[cvssv3_numerical_col].corr()
plt.figure(figsize=(24,8))
sns.heatmap(cvss3_df[cvssv3_numerical_col].corr(), annot=True, fmt=".2f");

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
cvss3_df.describe()
cvss3_df.head

# Prepare the features for modelling

In [ ]:

cvssv3_model_df= cvss3_df[cvssv3_categorical_col]
cvssv3_model_df.columns

#cvssv3_model_df.info

In [ ]:
cvssv3_model_df.isnull().any()

In [ ]:
cvssv3_categorical_col

In [ ]:
cvssv3_categorical_col.remove('cvssV3_baseSeverity')


# Perform One Hot Encoding
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [ ]:
df_v3_model = pd.get_dummies(cvssv3_model_df[cvssv3_categorical_col])


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_v3_model.describe()
df_v3_model.info

In [ ]:
plt.figure(figsize=(24,8))
sns.heatmap(df_v3_model.corr(), annot=True, fmt=".2f");

In [ ]:
import numpy as np

# Create correlation matrix
corr_matrix = df_v3_model.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]


# Drop features 
df_v3_model.drop(to_drop, axis=1, inplace=True)
to_drop

In [ ]:
#X = pd.concat([df['cvssV3_baseScore'], df_v3_model], axis=1)
X = df_v3_model
X.shape
#X.columns

In [ ]:
plt.figure(figsize=(24,8))

# Mask for the upper triangle
mask = np.zeros_like(X.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 220, as_cmap=True)
cmap = sns.light_palette((210, 90, 60), input="husl")

# Heatmap with mask
sns.heatmap(X.corr(), mask=mask, cmap=cmap, annot=True, fmt=".2f");

In [ ]:
correlatedColumns = []
corr = X.corr()
indices = corr.index
columns = corr.columns
posthreshold = 0.7
negthreshold = -0.7

for c in columns:
    for r in indices:
        if c != r and (corr[c][r] > posthreshold or corr[c][r] < negthreshold):
            print("column "  + c , "  row " + r + "  val " + str(corr[c][r]) )
            correlatedColumns.append({"column" : c , "row" : r , "val" :corr[c][r] })
            

print(correlatedColumns)

In [ ]:
cvssv3_model_df.columns
cvssv3_model_df.cvssV3_privilegesRequired.unique
cvssv3_model_df.cvssV3_integrityImpact.unique

# Prepare the features with basic categorization

In [ ]:
def calculateSeverityScore(row):
    #baseScore = score
    if (row['cvssV3_attackVector'] == "NETWORK" and row['cvssV3_availabilityImpact'] == "HIGH" and row['cvssV3_integrityImpact'] == "HIGH" and row['cvssV3_privilegesRequired'] == "NONE" and row['cvssV3_userInteraction'] == "NONE"):
        return 'CRITICAL'
    elif (row['cvssV3_attackVector'] == 'NETWORK' and row['cvssV3_availabilityImpact'] == 'HIGH' and row['cvssV3_integrityImpact'] == 'HIGH' and row['cvssV3_privilegesRequired'] == 'NONE' and row['cvssV3_userInteraction'] == 'REQUIRED'):
        return 'CRITICAL'
    elif (row['cvssV3_attackVector'] == 'NETWORK' and row['cvssV3_availabilityImpact'] == 'HIGH' and row['cvssV3_integrityImpact'] == 'HIGH' and row['cvssV3_privilegesRequired'] != 'NONE'):
        return 'HIGH'
    elif (row['cvssV3_attackVector'] == 'NETWORK' and row['cvssV3_integrityImpact'] == 'HIGH' and row['cvssV3_privilegesRequired'] == 'NONE'):
        return 'HIGH'
    elif (row['cvssV3_attackVector'] == 'NETWORK' and row['cvssV3_integrityImpact'] == 'HIGH' and row['cvssV3_privilegesRequired'] != 'NONE'):
        return 'HIGH'
    elif (row['cvssV3_availabilityImpact'] != 'HIGH' and row['cvssV3_confidentialityImpact'] != 'LOW' and row['cvssV3_userInteraction'] == 'NONE' and row['cvssV3_integrityImpact'] != 'HIGH' and row['cvssV3_privilegesRequired'] == 'NONE'):
        return 'HIGH'
    elif (row['cvssV3_confidentialityImpact'] != 'LOW' and row['cvssV3_userInteraction'] == 'REQUIRED' and row['cvssV3_integrityImpact'] != 'HIGH' and row['cvssV3_privilegesRequired'] == 'NONE'):
        return 'LOW'        
    elif (row['cvssV3_confidentialityImpact'] == 'LOW' and row['cvssV3_integrityImpact'] != 'HIGH' and row['cvssV3_privilegesRequired'] == 'NONE'):
        return 'LOW'         
    elif (row['cvssV3_confidentialityImpact'] != 'LOW' and row['cvssV3_integrityImpact'] != 'HIGH' and row['cvssV3_privilegesRequired'] != 'NONE'):
        return 'LOW'            
    elif (row['cvssV3_confidentialityImpact'] != 'LOW' and row['cvssV3_integrityImpact'] != 'HIGH' and row['cvssV3_privilegesRequired'] != 'NONE'):
        return 'LOW'
    else:
        return 'LOW'

#severity = calculateSeverityScore(cvss3_df.iloc[0]['cvssV3_baseScore'])
#severity

cvssv3_model_df['Severity_Score'] = cvssv3_model_df.apply(lambda row: calculateSeverityScore(row),axis=1)


In [ ]:
cvssv3_model_df['Severity_Score'].unique()

In [ ]:
severity_to_num = {'CRITICAL': 1,
                    'HIGH': 2,
                    'LOW': 3}
cvssv3_model_df['Severity_Score_Num'] = cvssv3_model_df['Severity_Score'].map(severity_to_num)

In [ ]:
cvssv3_model_df['Severity_Score_Num'].unique()


In [ ]:
cvssv3_model_df.rename(columns = {'Severity_Score':'Severity_Score_Text'}, inplace = True)


In [ ]:
cvssv3_model_df.columns

In [ ]:
cvssv3_model_df.rename(columns = {'Severity_Score_Num':'Severity_Score'}, inplace = True)
Y = cvssv3_model_df['Severity_Score']
Y.head


In [ ]:
X.to_csv("features.csv", index=False, encoding='utf8')


In [ ]:
Y.to_csv("classes.csv",index=False,encoding='utf8')

In [ ]:
X.head

In [ ]:
Y.head

# Split the data set into 70% training and 30% test 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
clf.fit(X_train, y_train)



In [ ]:
from sklearn.tree.export import export_text
tree_rules = export_text(clf, feature_names = list(X.columns))
tree_rules

In [ ]:
def get_code(tree, feature_names):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]
        value = tree.tree_.value

        def recurse(left, right, threshold, features, node):
                if (threshold[node] != -2):
                        print ("if ( " + features[node] + " <= " + str(threshold[node]) + " ) {")
                        if left[node] != -1:
                                recurse (left, right, threshold, features,left[node])
                        print ("} else {")
                        if right[node] != -1:
                                recurse (left, right, threshold, features,right[node])
                        print ("}")
                else:
                        print ("return " + str(value[node]))

        recurse(left, right, threshold, features, 0)



In [ ]:
    get_code(clf,list(X.columns))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy_score(y_train, clf.predict(X_train))

In [ ]:
print(classification_report(y_train, clf.predict(X_train)))

In [ ]:
confusion_matrix(y_train, clf.predict(X_train))

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

In [ ]:
print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
confusion_matrix(y_test, clf.predict(X_test))

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        res = clf.predict(X_train)
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, 
                                                                res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, 
                                                                            res)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, 
                                                                  res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                      lb.transform(res))))

        #res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, 
                                                                res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, 
                                                                            res_test)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, 
                                                                  res_test)))   
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), 
                                                      lb.transform(res_test))))

In [ ]:
print_score(clf, X_train, X_test, y_train, y_test, train=True)
print_score(clf, X_train, X_test, y_train, y_test, train=False)

In [ ]:
from sklearn import tree

text_representation = tree.export_text(clf)
print(text_representation)

In [ ]:
def my_tokenizer(text):
    return text.split(':') if text != None else []

In [ ]:
len(Text_df)

In [ ]:
Text_df = pd.read_csv("D:\\repos\\APDSProject\\MLTuning\\datacollection\\CWE-Base.csv")
Text_df.shape
#Text_df.columns
#Text_df.dtypes

In [ ]:
text_df_new = Text_df[Text_df['Common Consequences'].notnull()]
text_df_new.shape

In [ ]:
tokens = text_df_new['Common Consequences'].map(my_tokenizer).sum()

In [ ]:
#tokens.remove('SCOPE')
tokens = [ elem for elem in tokens if elem != 'SCOPE']
tokens = [ elem for elem in tokens if elem != 'IMPACT']
tokens = [ elem for elem in tokens if elem != 'NOTE']
tokens = [ elem for elem in tokens if elem != 'Access Control']
tokens = [ elem for elem in tokens if elem != 'Authentication']
tokens = [ elem for elem in tokens if elem != 'Availability']
tokens = [ elem for elem in tokens if elem != 'Confidentiality']
tokens = [ elem for elem in tokens if elem != 'Integrity']
tokens = [ elem for elem in tokens if elem != 'Non-Repudiation']
tokens = [ elem for elem in tokens if elem != 'Other']

In [ ]:
from collections import Counter

counter = Counter(tokens)
counter.most_common(30)


In [ ]:
Text_df['CWE-ID'].head

In [ ]:
def checkConsequence(text):
    check = Text_df[Text_df['Common Consequences'].str.contains('Crash, Exit, or Restart')